In [1]:
import os
import pandas as pd

In [2]:
def import_data(file_path, cols, index_col="STU_ID", low_memory=False):
    
    df = pd.read_csv(file_path, index_col=index_col, usecols=cols, low_memory=low_memory)
    
    return df

## Import Data

In [3]:
els_file_path = os.path.join("data", "ELS-2002", "els_02_12_byf3pststu_v1_0.csv")

# only keep these columns
cols = [
    "STU_ID", # index
    "BYSEX", # parameters to input into model start
    "BYRACE",
    "BYSTLANG",
    "BYPARED",
    "BYINCOME",
    "BYURBAN",
    "BYREGION",
    "BYRISKFC",
    "BYS34A",
    "BYS34B",
    "BYWRKHRS",
    "BYS42",
    "BYS43",
    "BYTVVIGM",
    "BYS46B",
    "BYS44C",
    "BYS20E",
    "BYS87C",
    "BYS20D",
    "BYS23C",
    "BYS37",
    "BYS27I",
    "BYS90D",
    "BYS38A",
    "BYS20J",
    "BYS24C",
    "BYS24D",
    "BYS54I",
    "BYS84D",
    "BYS84I",
    "BYS85A", # parameters to input into model end
    "F2HSSTAT", # parameters to predict start
    "F2EVERDO",
    "F1RGPP2" # parameters to predict end
    ]

df = import_data(els_file_path, cols)
cols.remove("STU_ID")
df = df[cols]
df

,BYSEX,BYRACE,BYSTLANG,BYPARED,BYINCOME,BYURBAN,BYREGION,BYRISKFC,BYS34A,BYS34B,...,BYS20J,BYS24C,BYS24D,BYS54I,BYS84D,BYS84I,BYS85A,F2HSSTAT,F2EVERDO,F1RGPP2
STU_ID,,,,,,,,,,,,,,,,,,,,,
101101,2,5,1,5,10,1,1,2,1,6,...,3,3,1,3,1,1,4,1,0,2
101102,2,2,0,5,11,1,1,0,1,4,...,3,2,1,2,1,1,4,1,0,4
101104,2,7,1,2,10,1,1,-9,-9,18,...,-9,2,1,2,1,1,4,1,0,4
101105,2,3,1,2,2,1,1,-4,4,7,...,-9,3,1,1,1,1,4,1,0,4
101106,2,4,0,1,6,1,1,2,8,2,...,3,2,1,3,1,1,4,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461230,2,4,0,2,9,1,3,-4,4,0,...,3,2,1,3,0,1,1,3,0,2
461231,2,4,0,3,3,1,3,5,5,1,...,4,3,1,3,0,1,3,8,1,3
461232,2,5,0,1,5,1,3,3,1,3,...,3,3,1,2,1,1,4,8,1,2


# Clean data

### Anyone meeting the following criteria is considered a 1 (i.e., will graduate high school or earn equivalent)
* high school graduate
* enrolled in high school or working towards GED
* received GED or certificate of attendance

### Anyone meeting the following criteria is considered a 0 (i.e., will not graduate high school or earn equivalent)
* no diploma, not in high school, not pursuing GED
* status undetermined

### Reorganize and change bin structure for remaining data

In [4]:
df = (
    df
    .assign(F2HSSTAT=lambda df: df["F2HSSTAT"].replace(to_replace=[1, 2, 3, 4, 5, 6, 7, 8], value=1)) # received HS diploma or equivalent
    .assign(F2HSSTAT=lambda df: df["F2HSSTAT"].replace(to_replace=[9, 10], value=0)) # not received HS diploma or equivalent
    
    .assign(F1RGPP2=lambda df: df["F1RGPP2"].replace(to_replace=[0, 1], value=0)) # bin GPA values to 0 (D or F)
    .assign(F1RGPP2=lambda df: df["F1RGPP2"].replace(to_replace=[2, 3], value=1)) # bin GPA values to 1 (C)
    .assign(F1RGPP2=lambda df: df["F1RGPP2"].replace(to_replace=[4, 5], value=2)) # bin GPA values to 2 (B)
    .assign(F1RGPP2=lambda df: df["F1RGPP2"].replace(to_replace=6, value=3)) # bin GPA values to 3 (A)

    .assign(BYSEX=lambda df: df["BYSEX"].replace(to_replace=2, value=0)) # change female to category 0

    .assign(BYRACE=lambda df: df["BYRACE"].replace(to_replace=[1, 2, 3, 4, 5, 6], value=0)) # combine all non-White ethnicities
    .assign(BYRACE=lambda df: df["BYRACE"].replace(to_replace=7, value=1)) # move White race to  category 5
    
    .assign(BYPARED=lambda df: df["BYPARED"].replace(to_replace=1, value=0)) # parents did not graduate high school
    .assign(BYPARED=lambda df: df["BYPARED"].replace(to_replace=[2, 3, 4, 5, 6, 7, 8], value=1)) # combine all parents that graduated high school

    .assign(BYINCOME=lambda df: df["BYINCOME"].replace(to_replace=[1, 2, 3, 4, 5, 6, 7], value=0)) # combine household income categories <= $25k
    .assign(BYINCOME=lambda df: df["BYINCOME"].replace(to_replace=[8, 9, 10, 11, 12, 13], value=1)) # combine household income categories greater than $25k
    
    .assign(BYURBAN=lambda df: df["BYURBAN"].replace(to_replace=[1, 2], value=0)) # Convert urban and suburban to not rural
    .assign(BYURBAN=lambda df: df["BYURBAN"].replace(to_replace=3, value=1)) # Renumber rural
    
    .assign(BYRISKFC=lambda df: df["BYRISKFC"].replace(to_replace=[1, 2, 3, 4, 5], value=1)) # Combine all risk factors
    
    .assign(BYS34A=lambda df: df["BYS34A"].replace(to_replace=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], value=1)) # bin "Hours/week spent on homework in school" into 10-hour categories
    .assign(BYS34A=lambda df: df["BYS34A"].replace(to_replace=[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21], value=2))
    
    .assign(BYS34B=lambda df: df["BYS34B"].replace(to_replace=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], value=1))
    .assign(BYS34B=lambda df: df["BYS34B"].replace(to_replace=[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26], value=2))
    
    .assign(BYWRKHRS=lambda df: df["BYWRKHRS"].replace(to_replace=[1, 2, 3, 4, 5, 6, 7, 8, 9], value=1)) # if student worked at all, convert to category 1
    
    .assign(BYS42=lambda df: df["BYS42"].replace(to_replace=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], value=1)) # bin "Hours/week spent on extracurricular activities" into 10-hour categories
    .assign(BYS42=lambda df: df["BYS42"].replace(to_replace=[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21], value=2))
    
    .assign(BYS43=lambda df: df["BYS43"].replace(to_replace=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], value=1)) # bin "Hours/week spent reading outside of school" into 10-hour categories
    .assign(BYS43=lambda df: df["BYS43"].replace(to_replace=[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21], value=2))
    
    .assign(BYTVVIGM=lambda df: df["BYTVVIGM"].replace(to_replace=[0, 1, 2, 3, 4], value=0)) # bin "hrs/day watching TV/videos/playing video games (weekdays)" into 4-hour categories
    .assign(BYTVVIGM=lambda df: df["BYTVVIGM"].replace(to_replace=[5, 6, 7, 8], value=1))
    
    .assign(BYS46B=lambda df: df["BYS46B"].replace(to_replace=[0, 1, 2, 3], value=0)) # bin "Hours/day on computer other than for school" into 3-hour categories
    .assign(BYS46B=lambda df: df["BYS46B"].replace(to_replace=[4, 5, 6], value=1))
)

### Convert all missing, unknown, skipped, nonrespondent data to value of -1. These values will be imputed using scikitlearn.

In [5]:
for col in df:
    df.loc[(df[col] < 0) | (df[col] == 99), col] = -1

### Remove data with missing GPA values

In [6]:
df = df[df["F1RGPP2"] != -1]
df

,BYSEX,BYRACE,BYSTLANG,BYPARED,BYINCOME,BYURBAN,BYREGION,BYRISKFC,BYS34A,BYS34B,...,BYS20J,BYS24C,BYS24D,BYS54I,BYS84D,BYS84I,BYS85A,F2HSSTAT,F2EVERDO,F1RGPP2
STU_ID,,,,,,,,,,,,,,,,,,,,,
101101,0,0,1,1,1,0,1,1,1,1,...,3,3,1,3,1,1,4,1,0,1
101102,0,0,0,1,1,0,1,0,1,1,...,3,2,1,2,1,1,4,1,0,2
101104,0,1,1,1,1,0,1,-1,-1,2,...,-1,2,1,2,1,1,4,1,0,2
101105,0,0,1,1,0,0,1,-1,1,1,...,-1,3,1,1,1,1,4,1,0,2
101106,0,0,0,0,0,0,1,1,1,1,...,3,2,1,3,1,1,4,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461230,0,0,0,1,1,0,3,-1,1,0,...,3,2,1,3,0,1,1,1,0,1
461231,0,0,0,1,0,0,3,1,1,1,...,4,3,1,3,0,1,3,1,1,1
461232,0,0,0,0,0,0,3,1,1,1,...,3,3,1,2,1,1,4,1,1,1


In [7]:
df.to_csv("clean_student_data.csv")